# CS 1656 – Introduction to Data Science 

## Instructor: Alexandros Labrinidis
## Teaching Assistants: Evangelos Karageorgos, Xiaoting Li, Gordon Lu
### Additional credits: Xiaoting Li, Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham
## Recitation : SQL via Data API
---
In this recitation, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [1]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [2]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 270 days ago)
# end_date = datetime.now()
# start_date = end_date - timedelta(days=270)

# Get two date endpoints
start_date = date(2018, 9, 1)
end_date = date(2019, 6, 1)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}'""".format(resource_id, start_str, end_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

# Parse response JSON into python dictionary
response_data = json.loads(response.text)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")
print(df.columns)
df.head()


/Users/liam/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


19234 rows total
Index(['inspect_dt', 'rating', 'bus_st_date', 'high', 'street', 'id',
       'municipal', 'city', 'zip', 'state', 'low', 'facility_name', 'medium',
       'description', 'start_time', 'placard_st', 'description_new',
       'encounter', 'url', '_full_text', 'num', 'end_time', '_id'],
      dtype='object')


,inspect_dt,rating,bus_st_date,high,street,id,municipal,city,zip,state,...,description,start_time,placard_st,description_new,encounter,url,_full_text,num,end_time,_id
0,2018-09-04,V,2017-03-29,None,Alpha Drive,201704040001,O'Hara,Pittsburgh,15238,PA,...,Food Processor,14:10:00,1,Floors,201809040053,http://appsrv.alleghenycounty.us/reports/rwser...,'-03':9 '-04':22 '-09':21 '-29':10 '/reports/r...,615,15:30:00,176362
1,2018-09-04,V,2009-11-04,None,Wood St,200911120004,Wilkinsburg,Pittsburgh,15221,PA,...,Restaurant without Liquor,14:00:00,1,Floors,201809040043,http://appsrv.alleghenycounty.us/reports/rwser...,"'-04':10,23 '-09':22 '-11':9 '/reports/rwservl...",902,16:00:00,160958
2,2018-09-04,V,2005-09-26,F,Rochester Road,200510120001,Ross,Pittsburgh,15229,PA,...,Chain Supermarket,13:15:00,1,Cleaning and Sanitization,201809040038,http://appsrv.alleghenycounty.us/reports/rwser...,"'-04':24 '-09':9,23 '-26':10 '/reports/rwservl...",184,15:35:00,160935
3,2018-09-04,V,2005-09-26,F,Rochester Road,200510120001,Ross,Pittsburgh,15229,PA,...,Chain Supermarket,13:15:00,1,Handwashing Facilities,201809040038,http://appsrv.alleghenycounty.us/reports/rwser...,"'-04':23 '-09':9,22 '-26':10 '/reports/rwservl...",184,15:35:00,160936
4,2018-09-04,V,2005-09-26,F,Rochester Road,200510120001,Ross,Pittsburgh,15229,PA,...,Chain Supermarket,13:15:00,1,"Fabrication, Design, Installation and Maintenance",201809040038,http://appsrv.alleghenycounty.us/reports/rwser...,"'-04':26 '-09':9,25 '-26':10 '/reports/rwservl...",184,15:35:00,160937


Details of dataset attributes are below. ((Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)
![title](attribs.png)

## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the time span (violation description[violation]).__

In [3]:
query = """
SELECT DISTINCT "description_new" as violation
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' """.format(resource_id, start_str, end_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/Users/liam/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,violation
0,General Premises
1,Lighting
2,Cold Holding Temperatures
3,Toilet Room
4,Consumer Advisory
5,Garbage and Refuse
6,Toxic Items
7,Cleaning and Sanitization
8,Demonstration of Knowledge
9,Reheating Temperatures


__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category (facility name[facility], number of violations[count]).__

In [4]:
query = """
SELECT "facility_name" as facility, COUNT("description_new") as count
FROM "{}"
WHERE "inspect_dt" BETWEEN '{}' and '{}' AND "city" = '{}' AND "rating" <> '{}'
GROUP BY "facility_name" """.format(resource_id, start_str, end_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query}, verify=False)

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

/Users/liam/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.wprdc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,count,facility
0,1,3 Lakes Golf Course 19th Hole
1,1,Apollo Cafe
2,2,Atria's Restaurant & Tavern
3,1,Bae Bae's Kitchen
4,1,Barb's Country Kitchen
...,...,...
119,1,TW Kitchen
120,1,Wendy's Old Fashioned Hamburgers #521
121,1,West View Firemen's Building
122,1,Wilson's Pharmacy


### Tasks

__Tasks 1 to 4 must be implemented in Task.py__

__Task 5 must be implemented in the notebook__

__T1) Find top 30 restaurants in Pittsburgh with maximum number of violations (facility name[facility], number of violations[count]).__

Lets look more closely into the inspection results of Pitt facilities.

__T2) Find the category descriptions and their high, medium, low risk ratings for all violations at facilities that start with 'Pitt' over the past nine months (facility name[facility], violation description[violation], rating[rating], high[high], medium[medium], low[low]).__

Now lets look at all facilities that contain word 'Pitt'.

__T3) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included (facility name[facility], violation description[violation], rating[rating], high[high], medium[medium], low[low])__

__T4) Find top 20 facilities that have word 'Pitt' in their name and have the highest counts of violations (facility name[facility], number of violations[count]).__